<a href="https://colab.research.google.com/github/ManviNarang01/AgroInsight/blob/main/RainPrediction(Part1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Algorithms used: KNN, DNN, ANN, Neural Network with 3 and 5 layers.**

**K-NEAREST NEIGHBOUR REGRESSION
(MSE: 0.4276)**

In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer

In [4]:
# Loading dataset
data_knn = pd.read_csv('weather.csv')

<ipython-input-4-f5567185c480>:2: DtypeWarning: Columns (1,2,3) have mixed types. Specify dtype option on import or set low_memory=False.
  data_knn = pd.read_csv('weather.csv')


In [5]:
# Data Preprocessing
data_knn = data_knn.drop(columns=['time'])

# Separating features as (X_knn) and target variable as (y_knn)
X_knn = data_knn.drop(columns=['rain (mm)'])
y_knn = data_knn['rain (mm)']

In [6]:
# Correcting missing values in the target variable
y_knn = y_knn.fillna(0)

In [7]:
# Splitting the data into training and testing sets
X_train_knn, X_test_knn, y_train_knn, y_test_knn = train_test_split(X_knn, y_knn, test_size=0.2, random_state=42)

In [8]:
# Preprocessing further
numeric_features_knn = X_knn.select_dtypes(include=['float64']).columns
numeric_transformer_knn = make_pipeline(
    SimpleImputer(strategy='mean'),
    StandardScaler()
)

preprocessor_knn = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer_knn, numeric_features_knn)
    ])

In [9]:
# Creating the model pipeline
model_knn = make_pipeline(preprocessor_knn, KNeighborsRegressor())

In [10]:
# Training the model
model_knn.fit(X_train_knn, y_train_knn)

# Making predictions on the test set
y_pred_knn = model_knn.predict(X_test_knn)

In [11]:
# Evaluating the model usingMSE
mse_knn = mean_squared_error(y_test_knn, y_pred_knn)
print(f'Mean Squared Error (KNN): {mse_knn}')

Mean Squared Error (KNN): 0.4276556005693639


In [12]:
prediction_data = pd.read_csv('predictions.csv')
predictions_knn = model_knn.predict(prediction_data)

print("Predicted Rain for the next 3 months:")
print(predictions_knn)

Predicted Rain for the next 3 months:
[0.06 0.   0.   ... 0.   0.04 0.04]


**NEURAL NETWORKS: 3 LAYER (MSE:0.3880)**

In [25]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import make_pipeline
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [26]:
# Loading dataset
data = pd.read_csv('weather.csv')

# Data Preprocessing
data = data.drop(columns=['time'])

# checking data types of all columns
print("Data Types of Columns:")
print(data.dtypes)

Data Types of Columns:
temperature_2m (°C)                   object
relative_humidity_2m (%)              object
precipitation (mm)                    object
rain (mm)                            float64
wind_speed_100m (km/h)               float64
soil_temperature_28_to_100cm (°C)    float64
soil_moisture_7_to_28cm (m³/m³)      float64
is_day ()                            float64
dtype: object


<ipython-input-26-01a253b57a82>:2: DtypeWarning: Columns (1,2,3) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('weather.csv')


In [27]:
# Separating features as (X) and target variable as (y)
X = data.drop(columns=['rain (mm)'])
y = data['rain (mm)']

# Correcting missing values in the target variable
y = y.fillna(0)

In [28]:
# Splitting the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Further preprocessing
numeric_features = X.select_dtypes(include=['float64']).columns
numeric_transformer = make_pipeline(
    SimpleImputer(strategy='mean'),
    StandardScaler()
)

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features)
    ])

In [29]:
preprocessor.fit(X_train)

ColumnTransformer(transformers=[('num',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer()),
                                                 ('standardscaler',
                                                  StandardScaler())]),
                                 Index(['wind_speed_100m (km/h)', 'soil_temperature_28_to_100cm (°C)',
       'soil_moisture_7_to_28cm (m³/m³)', 'is_day ()'],
      dtype='object'))])

In [30]:
# Displaying columns before and after preprocessing
print("\nColumns before preprocessing:")
print(X_train.columns)


Columns before preprocessing:
Index(['temperature_2m (°C)', 'relative_humidity_2m (%)', 'precipitation (mm)',
       'wind_speed_100m (km/h)', 'soil_temperature_28_to_100cm (°C)',
       'soil_moisture_7_to_28cm (m³/m³)', 'is_day ()'],
      dtype='object')


In [31]:
# Creating the neural network model
model_nn = Sequential([
    Dense(64, activation='relu', input_shape=(len(numeric_features),)),
    Dense(32, activation='relu'),
    Dense(1)  # Output layer with 1 neuron for regression
])

# Compiling the model
model_nn.compile(optimizer='adam', loss='mean_squared_error')

In [32]:
# Displaying model summary
model_nn.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 64)                320       
                                                                 
 dense_4 (Dense)             (None, 32)                2080      
                                                                 
 dense_5 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2433 (9.50 KB)
Trainable params: 2433 (9.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [33]:
# Training the model
model_nn.fit(preprocessor.transform(X_train), y_train, epochs=50, batch_size=32, validation_split=0.2, verbose=1)

Epoch 1/50
1827/1827 [==============================] - 5s 2ms/step - loss: 0.4072 - val_loss: 0.4476
Epoch 2/50
1827/1827 [==============================] - 6s 3ms/step - loss: 0.3876 - val_loss: 0.4326
Epoch 3/50
1827/1827 [==============================] - 4s 2ms/step - loss: 0.3768 - val_loss: 0.4219
Epoch 4/50
1827/1827 [==============================] - 4s 2ms/step - loss: 0.3705 - val_loss: 0.4211
Epoch 5/50
1827/1827 [==============================] - 6s 3ms/step - loss: 0.3647 - val_loss: 0.4156
Epoch 6/50
1827/1827 [==============================] - 4s 2ms/step - loss: 0.3626 - val_loss: 0.4073
Epoch 7/50
1827/1827 [==============================] - 6s 3ms/step - loss: 0.3581 - val_loss: 0.4149
Epoch 8/50
1827/1827 [==============================] - 8s 4ms/step - loss: 0.3600 - val_loss: 0.4113
Epoch 9/50
1827/1827 [==============================] - 7s 4ms/step - loss: 0.3568 - val_loss: 0.4200
Epoch 10/50
1827/1827 [==============================] - 7s 4ms/step - loss: 0.355

In [34]:
# Making predictions on the test set
y_pred_nn = model_nn.predict(preprocessor.transform(X_test))

571/571 [==============================] - 1s 1ms/step


In [35]:
# Evaluating the model
mse_nn = mean_squared_error(y_test, y_pred_nn)

print(f'\nMean Squared Error (Neural Network): {mse_nn}')


Mean Squared Error (Neural Network): 0.3880342382168559


**DEEP NEURAL NETWORK : 3 LAYERS (MSE:0.395)**

In [47]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import make_pipeline
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam

In [48]:
#Loading dataset
data_dnn = pd.read_csv('weather.csv')

# Data Preprocessing
data_dnn = data_dnn.drop(columns=['time'])

# Separating features as (X_dnn) and target variable as (y_dnn)
X_dnn = data_dnn.drop(columns=['rain (mm)'])
y_dnn = data_dnn['rain (mm)']

#Correcting missing values in the target variable
y_dnn = y_dnn.fillna(0)  # You can use a different strategy to handle missing values, e.g., mean, median, etc.

# Splitting the data into training and testing sets
X_train_dnn, X_test_dnn, y_train_dnn, y_test_dnn = train_test_split(X_dnn, y_dnn, test_size=0.2, random_state=42)

# Further preprocessing
numeric_features_dnn = X_dnn.select_dtypes(include=['float64']).columns
numeric_transformer_dnn = make_pipeline(
    SimpleImputer(strategy='mean'),
    StandardScaler()
)

preprocessor_dnn = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer_dnn, numeric_features_dnn)
    ])

preprocessor_dnn.fit(X_train_dnn[numeric_features_dnn])

<ipython-input-48-a4046743a1d0>:2: DtypeWarning: Columns (1,2,3) have mixed types. Specify dtype option on import or set low_memory=False.
  data_dnn = pd.read_csv('weather.csv')


ColumnTransformer(transformers=[('num',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer()),
                                                 ('standardscaler',
                                                  StandardScaler())]),
                                 Index(['wind_speed_100m (km/h)', 'soil_temperature_28_to_100cm (°C)',
       'soil_moisture_7_to_28cm (m³/m³)', 'is_day ()'],
      dtype='object'))])

In [49]:
#Creating the model
model_dnn = Sequential()
model_dnn.add(Dense(64, activation='relu', input_shape=(len(numeric_features_dnn),)))  # Update input_shape
model_dnn.add(Dense(32, activation='relu'))
model_dnn.add(Dense(1, activation='linear'))  # Regression output layer

# Compiling the model
model_dnn.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')


In [50]:
#Transforming the training data
X_train_dnn_transformed = preprocessor_dnn.transform(X_train_dnn)

model_dnn.fit(X_train_dnn_transformed, y_train_dnn, epochs=50, batch_size=32, validation_split=0.2, verbose=1)

#Transforming the test data
X_test_dnn_transformed = preprocessor_dnn.transform(X_test_dnn)

#Making predictions on the test set
y_pred_dnn = model_dnn.predict(X_test_dnn_transformed)

#Evaluating the model using MSE
mse_dnn = mean_squared_error(y_test_dnn, y_pred_dnn)
print(f'Mean Squared Error: {mse_dnn}')

Epoch 1/50
1827/1827 [==============================] - 5s 2ms/step - loss: 0.4064 - val_loss: 0.4475
Epoch 2/50
1827/1827 [==============================] - 6s 3ms/step - loss: 0.3864 - val_loss: 0.4366
Epoch 3/50
1827/1827 [==============================] - 4s 2ms/step - loss: 0.3760 - val_loss: 0.4205
Epoch 4/50
1827/1827 [==============================] - 4s 2ms/step - loss: 0.3681 - val_loss: 0.4384
Epoch 5/50
1827/1827 [==============================] - 6s 3ms/step - loss: 0.3655 - val_loss: 0.4316
Epoch 6/50
1827/1827 [==============================] - 4s 2ms/step - loss: 0.3621 - val_loss: 0.4124
Epoch 7/50
1827/1827 [==============================] - 4s 2ms/step - loss: 0.3589 - val_loss: 0.4275
Epoch 8/50
1827/1827 [==============================] - 6s 3ms/step - loss: 0.3573 - val_loss: 0.4115
Epoch 9/50
1827/1827 [==============================] - 4s 2ms/step - loss: 0.3571 - val_loss: 0.4073
Epoch 10/50
1827/1827 [==============================] - 4s 2ms/step - loss: 0.355

**ARTIFICIAL NEURAL NETWORK (MSE:0.393)**

In [59]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import make_pipeline
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam

# Loading the dataset
data_ann = pd.read_csv('weather.csv')

# Data Preprocessing
data_ann = data_ann.drop(columns=['time'])

# Separating features as  (X_ann) and target variable as (y_ann)
X_ann = data_ann.drop(columns=['rain (mm)'])
y_ann = data_ann['rain (mm)']

#Correcting missing values in the target variable
y_ann = y_ann.fillna(0)  # You can use a different strategy to handle missing values, e.g., mean, median, etc.

# Splitting the data into training and testing sets
X_train_ann, X_test_ann, y_train_ann, y_test_ann = train_test_split(X_ann, y_ann, test_size=0.2, random_state=42)

#Further Preprocessing
numeric_features_ann = X_ann.select_dtypes(include=['float64']).columns
numeric_transformer_ann = make_pipeline(
    SimpleImputer(strategy='mean'),
    StandardScaler()
)

preprocessor_ann = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer_ann, numeric_features_ann)
    ])

preprocessor_ann.fit(X_train_ann[numeric_features_ann])

<ipython-input-59-46810342ae7f>:13: DtypeWarning: Columns (1,2,3) have mixed types. Specify dtype option on import or set low_memory=False.
  data_ann = pd.read_csv('weather.csv')


ColumnTransformer(transformers=[('num',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer()),
                                                 ('standardscaler',
                                                  StandardScaler())]),
                                 Index(['wind_speed_100m (km/h)', 'soil_temperature_28_to_100cm (°C)',
       'soil_moisture_7_to_28cm (m³/m³)', 'is_day ()'],
      dtype='object'))])

In [60]:
#Creating the model
model_ann = Sequential()
model_ann.add(Dense(64, activation='relu', input_shape=(len(numeric_features_ann),)))
model_ann.add(Dense(32, activation='relu'))
model_ann.add(Dense(1, activation='linear'))  # Regression output layer

# Compiling the model
model_ann.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')

# Transforming the training data
X_train_ann_transformed = preprocessor_ann.transform(X_train_ann)

model_ann.fit(X_train_ann_transformed, y_train_ann, epochs=50, batch_size=32, validation_split=0.2, verbose=1)

# Transforming the test data
X_test_ann_transformed = preprocessor_ann.transform(X_test_ann)

# Making predictions on the test set
y_pred_ann = model_ann.predict(X_test_ann_transformed)

# Evaluating the model
mse_ann = mean_squared_error(y_test_ann, y_pred_ann)
print(f'Mean Squared Error: {mse_ann}')

Epoch 1/50
1827/1827 [==============================] - 5s 2ms/step - loss: 0.4065 - val_loss: 0.4512
Epoch 2/50
1827/1827 [==============================] - 6s 3ms/step - loss: 0.3878 - val_loss: 0.4392
Epoch 3/50
1827/1827 [==============================] - 4s 2ms/step - loss: 0.3760 - val_loss: 0.4266
Epoch 4/50
1827/1827 [==============================] - 4s 2ms/step - loss: 0.3700 - val_loss: 0.4257
Epoch 5/50
1827/1827 [==============================] - 6s 3ms/step - loss: 0.3648 - val_loss: 0.4116
Epoch 6/50
1827/1827 [==============================] - 4s 2ms/step - loss: 0.3612 - val_loss: 0.4084
Epoch 7/50
1827/1827 [==============================] - 4s 2ms/step - loss: 0.3599 - val_loss: 0.4053
Epoch 8/50
1827/1827 [==============================] - 6s 3ms/step - loss: 0.3585 - val_loss: 0.4091
Epoch 9/50
1827/1827 [==============================] - 4s 2ms/step - loss: 0.3555 - val_loss: 0.4048
Epoch 10/50
1827/1827 [==============================] - 7s 4ms/step - loss: 0.355

**NEURAL NETWORK: 5 LAYERS (MSE: 0.390)**

In [51]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import make_pipeline
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [52]:
# Loading the dataset
data = pd.read_csv('weather.csv')

#Data Preprocessing
data = data.drop(columns=['time'])

<ipython-input-52-3e72b9e36910>:2: DtypeWarning: Columns (1,2,3) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('weather.csv')


In [53]:
# Separating features as (X) and target variable as (y)
X = data.drop(columns=['rain (mm)'])
y = data['rain (mm)']

# Correcting missing values in the target variable
y = y.fillna(0)

In [54]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Further preprocessing
numeric_features = X.select_dtypes(include=['float64']).columns
numeric_transformer = make_pipeline(
    SimpleImputer(strategy='mean'),
    StandardScaler()
)

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features)
    ])

In [55]:
preprocessor.fit(X_train)

ColumnTransformer(transformers=[('num',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer()),
                                                 ('standardscaler',
                                                  StandardScaler())]),
                                 Index(['wind_speed_100m (km/h)', 'soil_temperature_28_to_100cm (°C)',
       'soil_moisture_7_to_28cm (m³/m³)', 'is_day ()'],
      dtype='object'))])

In [56]:
# Creating a deeper neural network model
model_nn = Sequential([
    Dense(128, activation='relu', input_shape=(len(numeric_features),)),
    Dense(64, activation='relu'),
    Dense(32, activation='relu'),
    Dense(16, activation='relu'),
    Dense(1)  # Output layer with 1 neuron for regression
])

In [57]:
# Compiling the model with a lower learning rate
model_nn.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), loss='mean_squared_error')

# Displaying model summary
model_nn.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_14 (Dense)            (None, 128)               640       
                                                                 
 dense_15 (Dense)            (None, 64)                8256      
                                                                 
 dense_16 (Dense)            (None, 32)                2080      
                                                                 
 dense_17 (Dense)            (None, 16)                528       
                                                                 
 dense_18 (Dense)            (None, 1)                 17        
                                                                 
Total params: 11521 (45.00 KB)
Trainable params: 11521 (45.00 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [58]:
# Training the model with 100 epochs
model_nn.fit(preprocessor.transform(X_train), y_train, epochs=100, batch_size=32, validation_split=0.2, verbose=1)

# Making predictions on the test set
y_pred_nn = model_nn.predict(preprocessor.transform(X_test))

# Evaluating the model
mse_nn = mean_squared_error(y_test, y_pred_nn)

print(f'\nMean Squared Error (Neural Network): {mse_nn}')

Epoch 1/100
1827/1827 [==============================] - 10s 4ms/step - loss: 0.4219 - val_loss: 0.4636
Epoch 2/100
1827/1827 [==============================] - 5s 3ms/step - loss: 0.4015 - val_loss: 0.4515
Epoch 3/100
1827/1827 [==============================] - 7s 4ms/step - loss: 0.3917 - val_loss: 0.4419
Epoch 4/100
1827/1827 [==============================] - 6s 3ms/step - loss: 0.3848 - val_loss: 0.4353
Epoch 5/100
1827/1827 [==============================] - 6s 3ms/step - loss: 0.3788 - val_loss: 0.4328
Epoch 6/100
1827/1827 [==============================] - 7s 4ms/step - loss: 0.3737 - val_loss: 0.4251
Epoch 7/100
1827/1827 [==============================] - 6s 3ms/step - loss: 0.3691 - val_loss: 0.4211
Epoch 8/100
1827/1827 [==============================] - 6s 4ms/step - loss: 0.3663 - val_loss: 0.4193
Epoch 9/100
1827/1827 [==============================] - 5s 3ms/step - loss: 0.3634 - val_loss: 0.4155
Epoch 10/100
1827/1827 [==============================] - 5s 3ms/step - 